In [1]:
import os
from dotenv import load_dotenv

# Dynamically get the path to the .env file for Jupyter/IPython
current_directory = os.getcwd()
env_path = os.path.join(current_directory, 'API_KEY.env')

# Load the .env file
load_dotenv(dotenv_path=env_path)

# Access API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
NINJA_API_KEY = os.getenv("NINJA_API_KEY")
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")


In [2]:
import requests
import pandas as pd
import yfinance as yf
import csv
import time
from datetime import datetime, timedelta

# Replace with your Alpha Vantage API key
ALPHA_VANTAGE_API_KEY = 'YOUR_ALPHA_VANTAGE_API_KEY'

# Tickers to process
tickers = [
    "AAPL", "MSFT",  # Information Technology
    "JNJ", "PFE",    # Healthcare
    "JPM", "BAC",    # Financials
    "AMZN", "TSLA",  # Consumer Discretionary
    "PG", "KO",      # Consumer Staples
    "XOM", "CVX",    # Energy
    "BA", "CAT",     # Industrials
    "LIN", "DOW",    # Materials
    "AMT", "SPG",    # Real Estate
    "NEE", "DUK",    # Utilities
    "GOOGL", "META"  # Communication Services
]

# Define the range of years
start_year = 2019
end_year = 2023

# CSV file to save price growth metrics
output_file = "price_growth_metrics.csv"

# Initialize the CSV file
with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["ticker", "earningsCallDate", "reportedEPS", "estimatedEPS", "EPS_Surprise",
                     "T-1_price", "T_price", "T+1_price",
                     "7_day_growth", "14_day_growth", "30_day_growth", "90_day_growth"])  # Header row

# Function to fetch earnings call dates and financials
def fetch_earnings_call_data(ticker):
    url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}"
    response = requests.get(url)
    print(f"Fetching earnings data for {ticker}...")
    if response.status_code == 200:
        data = response.json()
        print(f"Response for {ticker}: {data}")
        return data
    else:
        print(f"Failed to fetch earnings call data for {ticker}: {response.text}")
        return None

# Helper function to find the closest trading day
def get_closest_trading_day(hist, target_date):
    try:
        # Convert target_date to match the timezone of hist.index
        if hist.index.tz is not None:  # If historical data has a timezone
            if pd.Timestamp(target_date).tz is None:  # If target_date is timezone-naive
                target_date = pd.Timestamp(target_date).tz_localize(hist.index.tz)
            else:  # If target_date is already timezone-aware
                target_date = pd.Timestamp(target_date).tz_convert(hist.index.tz)
        
        # Find the closest date in the index
        available_dates = hist.index
        closest_date = min(available_dates, key=lambda x: abs(x - target_date))
        print(f"Closest trading day to {target_date}: {closest_date}")
        return closest_date
    except Exception as e:
        print(f"Error finding closest trading day: {e}")
        return None

# Function to calculate price growth and prices around the earnings date
def calculate_price_growth(ticker, earnings_date, intervals=[7, 14, 30, 90]):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=earnings_date - timedelta(days=5), end=earnings_date + timedelta(days=100))
        print(f"Historical data for {ticker} on {earnings_date}: {hist}")
        
        # Ensure earnings_date is timezone-aware if needed
        earnings_date = pd.Timestamp(earnings_date)
        if hist.index.tz is not None:  # Match timezone awareness
            earnings_date = earnings_date.tz_localize(hist.index.tz)
        
        earnings_date = get_closest_trading_day(hist, earnings_date)
        if earnings_date is None:
            return {
                "T-1_price": None,
                "T_price": None,
                "T+1_price": None,
                **{f"{interval}_day_growth": None for interval in intervals}
            }
        
        # Get T-1 (day before) and T+1 (day after) prices
        t_minus_1_date = get_closest_trading_day(hist, earnings_date - timedelta(days=1))
        t_plus_1_date = get_closest_trading_day(hist, earnings_date + timedelta(days=1))
        
        t_minus_1_price = hist.loc[t_minus_1_date]["Close"] if t_minus_1_date else None
        t_price = hist.loc[earnings_date]["Close"] if earnings_date in hist.index else None
        t_plus_1_price = hist.loc[t_plus_1_date]["Close"] if t_plus_1_date else None
        
        # Calculate growth for specified intervals
        growth = {}
        for interval in intervals:
            future_date = earnings_date + timedelta(days=interval)
            future_date = get_closest_trading_day(hist, future_date)
            if future_date:
                growth[f"{interval}_day_growth"] = (
                    (hist.loc[future_date]["Close"] - hist.loc[earnings_date]["Close"]) / hist.loc[earnings_date]["Close"]
                ) * 100
            else:
                growth[f"{interval}_day_growth"] = None
        
        return {
            "T-1_price": t_minus_1_price,
            "T_price": t_price,
            "T+1_price": t_plus_1_price,
            **growth
        }
    except Exception as e:
        print(f"Error fetching price data for {ticker} on {earnings_date}: {e}")
        return {
            "T-1_price": None,
            "T_price": None,
            "T+1_price": None,
            **{f"{interval}_day_growth": None for interval in intervals}
        }

# Main script to fetch data and calculate metrics
def process_tickers():
    for ticker in tickers:
        earnings_data = fetch_earnings_call_data(ticker)
        if earnings_data and "quarterlyEarnings" in earnings_data:
            print(f"Earnings data for {ticker}: {earnings_data['quarterlyEarnings']}")
            for record in earnings_data["quarterlyEarnings"]:
                fiscal_date = record.get("fiscalDateEnding")
                if not fiscal_date:
                    continue
                try:
                    earnings_date = datetime.strptime(fiscal_date, "%Y-%m-%d").date()
                    if earnings_date.year < start_year or earnings_date.year > end_year:
                        continue
                    reported_eps = float(record.get("reportedEPS", 0))
                    estimated_eps = float(record.get("estimatedEPS", 0))
                    eps_surprise = reported_eps - estimated_eps
                    
                    # Calculate price growth
                    growth = calculate_price_growth(ticker, earnings_date)
                    print(f"Growth for {ticker} on {earnings_date}: {growth}")
                    
                    # Save to CSV
                    with open(output_file, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            ticker, earnings_date, reported_eps, estimated_eps, eps_surprise,
                            growth.get("T-1_price"), growth.get("T_price"), growth.get("T+1_price"),
                            growth.get("7_day_growth"), growth.get("14_day_growth"),
                            growth.get("30_day_growth"), growth.get("90_day_growth")
                        ])
                    print(f"Saved data for {ticker} {earnings_date}")
                except Exception as e:
                    print(f"Error processing record for {ticker} {earnings_date}: {e}")
        else:
            print(f"No earnings data found for {ticker}")
        time.sleep(15)  # Pause to respect API rate limits

# Run the script
process_tickers()


Fetching earnings data for AAPL...
Response for AAPL: {'symbol': 'AAPL', 'annualEarnings': [{'fiscalDateEnding': '2024-09-30', 'reportedEPS': '6.75'}, {'fiscalDateEnding': '2023-09-30', 'reportedEPS': '6.12'}, {'fiscalDateEnding': '2022-09-30', 'reportedEPS': '6.11'}, {'fiscalDateEnding': '2021-09-30', 'reportedEPS': '5.62'}, {'fiscalDateEnding': '2020-09-30', 'reportedEPS': '3.27'}, {'fiscalDateEnding': '2019-09-30', 'reportedEPS': '2.98'}, {'fiscalDateEnding': '2018-09-30', 'reportedEPS': '2.97'}, {'fiscalDateEnding': '2017-09-30', 'reportedEPS': '2.3'}, {'fiscalDateEnding': '2016-09-30', 'reportedEPS': '2.0675'}, {'fiscalDateEnding': '2015-09-30', 'reportedEPS': '2.3'}, {'fiscalDateEnding': '2014-09-30', 'reportedEPS': '1.6075'}, {'fiscalDateEnding': '2013-09-30', 'reportedEPS': '1.415'}, {'fiscalDateEnding': '2012-09-30', 'reportedEPS': '1.5775'}, {'fiscalDateEnding': '2011-09-30', 'reportedEPS': '0.9875'}, {'fiscalDateEnding': '2010-09-30', 'reportedEPS': '0.54'}, {'fiscalDateEndi